In [1]:
from fixedwidth.fixedwidth import FixedWidth
from pprint import pprint
from pyCifp.definitions import bootstrap_def, air_heliport_def, main_parser_def, \
    cont_base_parser_def, cont_app_parser_def, section_description_def as sdef
import requests
import re
import json

In [2]:
def p_builder(deflist, debug=False):
    """
    Parser Builder
    [
        ('fieldname', 'length'),
    ]
    """
    parse_config = {}
    fields_used = {}
    next_start = 1  # Start position for the next record. Cue from the Parse::FixedLength perl module
    
    for field in deflist:
        if debug:
            print(field)
        pre_field_name, length = field
        
        if pre_field_name not in fields_used:  # MORA MORA MORA.... *sigh*
            fields_used[pre_field_name] = 1
            field_name = pre_field_name
        else:
            field_name = pre_field_name + '_{fserial}'.format(fserial=fields_used[pre_field_name])
            fields_used[pre_field_name] += 1
            
        parse_config[field_name] = {
            'required': True,
            'type': 'string',
            'start_pos': next_start,
            'length': length,
            'alignment': 'left',
            'padding': ' '
        }
        next_start += length
        
    return parse_config

def fw_parse(fwobj, line):
    """
    Because... fuck your extra steps?
    """
    fwobj.line = line
    return fwobj.data

def get_section(line):
    """
    Apparently subsections are in different locations for some records...
    So this will get the sections
    Returns: (section_code, subsection_code)
    """
    base_parser = FixedWidth(p_builder(bootstrap_def))
    air_heli_parser = FixedWidth(p_builder(air_heliport_def))
    
    base_parsed = fw_parse(base_parser, line)
    
    if base_parsed['SectionCode'] in ['H', 'P'] and base_parsed['SubSectionCode'] == '':
        ah_parsed = fw_parse(air_heli_parser, line)
        return (ah_parsed['SectionCode'], ah_parsed['SubSectionCode'])
    else:
        return (base_parsed['SectionCode'], base_parsed['SubSectionCode'])

def dict_recurse(d):
    """
    Sleuth out and build a parser in a recursive dict
    d == dict
    """
    working_dict = {}
    
    for k, v in d.items():
        if isinstance(v, dict):
            working_dict[k] = dict_recurse(v)
        if isinstance(v, list):
            working_dict[k] = FixedWidth(p_builder(v))
    return working_dict



In [3]:
# Parser Configs
main_parsers = dict_recurse(main_parser_def)
bc_parsers = dict_recurse(cont_base_parser_def)
ac_parsers = dict_recurse(cont_app_parser_def)

In [4]:
def has_main_parser(section, subsection, mpd):
    """
    Check if there's a main parser for the section and subsection
    """
    if subsection not in mpd[section] or not isinstance(mpd[section][subsection], FixedWidth):
        print('Seemingly no parser for this line...')
        return False
    return True

In [ ]:
with open('FAACIFP18', mode='r') as cifp:
    cifp_lines = cifp.read().splitlines()

genset = set()
gendct = {}
genlist = []
parsed_list = []

for i, l in enumerate(cifp_lines):
    parsed_line = {
        'record_type': '',
        'pc': 'Primary',
        'data': {}
    }
    
    if l[0] != 'S':
        # Header or some other shit, ignore
        continue

    section_code, subsection_code = get_section(l)
    
    parsed_line['record_type'] = sdef[section_code][subsection_code]
    
    main_parser = main_parsers[section_code][subsection_code]
    main_line = fw_parse(main_parser, l)
    
    parsed_line['data'] = main_line
    
    is_continuation_record = False

    if int(main_line.get('ContinuationRecordNumber', 0)) == 2:
        is_continuation_record = True
        base_parser = bc_parsers[section_code][subsection_code]
        base_line = fw_parse(base_parser, l)
        
        application_type = base_line['ApplicationType']
        app_parser = ac_parsers[section_code][subsection_code][application_type]
        app_line = fw_parse(app_parser, l)
        
    if is_continuation_record:
        parsed_line['pc'] = 'Continuation'
        parsed_line['data'] = app_line
    
    parsed_list.append(parsed_line)

In [29]:
for i in parsed_list:
    if i['data'].get('ICAOCode') == 'OZW':
        print(json.dumps(i, indent=4))

In [25]:
with open('parsed_dump.json', mode='w') as dumpfile:
    json.dump(obj=parsed_list, fp=dumpfile, indent=4)